In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import html
import pandas as pd
import datetime
import time
import random
import warnings
import ast

from tqdm import tqdm

pd.set_option('display.max_columns', None) 
pd.options.mode.chained_assignment = None

In [2]:
date_to_run = '20230605'
df = pd.read_csv(f'xe_{date_to_run}.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5817 entries, 0 to 5816
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5817 non-null   int64  
 1   internal_id              5817 non-null   int64  
 2   item_type                5817 non-null   object 
 3   transaction_type         5817 non-null   object 
 4   address                  5817 non-null   object 
 5   geo_lat                  5817 non-null   float64
 6   geo_lng                  5817 non-null   float64
 7   ad_group_id              752 non-null    float64
 8   levels                   5817 non-null   object 
 9   bedrooms                 5533 non-null   float64
 10  bathrooms                5259 non-null   float64
 11  construction_year        4725 non-null   float64
 12  display_golden_border    5817 non-null   bool   
 13  owner_logo               4451 non-null   object 
 14  price_per_unit_area     

In [4]:
df.describe(include = ['object', 'bool'])

,item_type,transaction_type,address,levels,display_golden_border,owner_logo,company_title,is_commercial,sponsored,title,title_abbreviation,price,price_with_abbreviation,price_per_square_meter,size_with_square_meter,date,extra_seo_info,image_alt,image_gallery,url,unique_group_url,extraction_date
count,5817,5817,5817,5817,5817,4451,4452,5817,5817,5817,5817,5817,5817,5815,5817,2612,5817,5817,5817,5817,752,5817
unique,1,1,234,61,2,574,572,2,1,419,419,293,252,70,285,39,1,4855,5146,5811,752,1
top,re_residence,LET.NORMAL,Αθήνα (Κολωνάκι),['1ος'],False,https://a2.xe.gr/LOGO/1622120851959.jpg,Golden Home,True,False,Διαμέρισμα 50 τ.μ.,Διαμέρισμα 50 τ.μ.,450 €,450 €,8 €/τ.μ.,50 τ.μ.,πριν από 2 εβδομάδες,Ενοικίαση κατοικίας,Ενοικίαση κατοικίας Αθήνα (Κολωνάκι) Διαμέρισμ...,[],https://www.xe.gr/property/d/enoikiaseis-katoi...,https://www.xe.gr/property/u/460563,2023-06-05
freq,5817,5817,240,1238,5361,314,314,4523,5817,287,287,324,328,812,289,569,5817,7,667,2,1,5817


In [5]:
df.describe()

,id,internal_id,geo_lat,geo_lng,ad_group_id,bedrooms,bathrooms,construction_year,price_per_unit_area,account_id
count,5.817000e+03,5.817000e+03,5817.000000,5817.000000,752.000000,5533.000000,5259.000000,4725.000000,5817.000000,5.724000e+03
mean,7.954165e+08,4.642377e+07,37.986579,23.739821,428654.784574,1.863546,1.194334,1976.267725,11.667010,1.208623e+06
std,7.014771e+07,2.096783e+06,0.019577,0.020725,52509.421310,1.229310,0.735632,16.023442,50.245628,7.223853e+05
min,5.765660e+05,2.009286e+07,37.948830,23.687081,55188.000000,1.000000,1.000000,1900.000000,0.000000,2.003400e+04
25%,8.045262e+08,4.657052e+07,37.971645,23.727020,429311.250000,1.000000,1.000000,1970.000000,8.000000,5.404198e+05
50%,8.125159e+08,4.699916e+07,37.986418,23.739514,446462.500000,2.000000,1.000000,1975.000000,9.000000,1.304044e+06
75%,8.143908e+08,4.711343e+07,38.000596,23.753899,455809.500000,2.000000,1.000000,1980.000000,12.000000,1.881380e+06
max,8.154663e+08,4.716398e+07,38.032707,23.789568,461166.000000,21.000000,16.000000,2023.000000,3205.000000,2.268898e+06


In [6]:
useless_columns = ['item_type', 'transaction_type', 'owner_logo', 'title', 
                   'price_with_abbreviation', 'extra_seo_info', 'image_alt', 
                   'image_gallery', 'unique_group_url', 'price_per_square_meter', 'display_golden_border',
                   'is_commercial', 'sponsored']

In [7]:
useful_columns = ['id', 'internal_id', 'title_abbreviation','address','price','price_per_unit_area',
                  'size_with_square_meter','construction_year','levels','bedrooms', 'bathrooms',  
                   'date', 'extraction_date', 'geo_lat', 'geo_lng','company_title', 'account_id','ad_group_id', 'url']

In [8]:
len(useless_columns)+len(useful_columns)

32

In [9]:
df_useful = df.loc[:, useful_columns]

In [10]:
df_useful[['dimos', 'perioxi']] = df_useful['address'].str.extract(r'([\w\s]+)(?:\(([\w\s]+)\))?')
df_useful['dimos'] = df_useful['dimos'].str.strip()

In [11]:
dimoi_df = pd.read_csv('dimoi.csv')
dimoi_dict = dimoi_df.set_index('Raw')['Fixed'].to_dict()
missing_dimoi = set(df_useful['dimos'].unique()) - set(dimoi_dict.keys())
if missing_dimoi: 
    warnings.warn(f'The following dimoi are new: {missing_dimoi}')
df_useful['dimos'] = df_useful['dimos'].map(dimoi_dict)

In [12]:
df_useful['type'] = df_useful['title_abbreviation'].str.extract("(\w+)")
df_useful['price'] = df_useful['price'].str.replace('[.,]', '', regex=True).str.extract(r'(\d+)').astype('float')
df_useful['area'] = df['size_with_square_meter'].str.replace('[.,]', '', regex=True).str.extract(r'(\d+)').astype('float')
df_useful['price_per_area'] = df_useful['price'] / df_useful['area']
df_useful['date'] = df_useful['date'].fillna('πριν από 2 μήνες')

In [13]:
df_useful['levels'] = df_useful['levels'].str.replace('Υπόγειο', '-1').str.replace('Ισόγειο', '0').str.replace('Υπερυψωμένο', '0.5').str.replace('Ημιυπόγειο', '-0.5').str.replace('Ημιώροφος', '0.5').str.replace('ος', '')
df_useful['level_list'] = df_useful['levels'].apply(ast.literal_eval).apply(lambda lst: [float(element) for element in lst])

def get_smallest_level(level_list):
    if not level_list: return None
    min_val = min(level_list)
    if min_val < 0:
        non_negatives = [num for num in level_list if num >= 0]
        if non_negatives:
            min_val = min(non_negatives)
    return min_val
df_useful['min_level'] = df_useful['level_list'].apply(get_smallest_level)
df_useful['level_length'] = df_useful['level_list'].apply(len)

In [14]:
df_useful.isnull().sum()

id                           0
internal_id                  0
title_abbreviation           0
address                      0
price                        0
price_per_unit_area          0
size_with_square_meter       0
construction_year         1092
levels                       0
bedrooms                   284
bathrooms                  558
date                         0
extraction_date              0
geo_lat                      0
geo_lng                      0
company_title             1365
account_id                  93
ad_group_id               5065
url                          0
dimos                        0
perioxi                    304
type                         0
area                         0
price_per_area               0
level_list                   0
min_level                    2
level_length                 0
dtype: int64

In [15]:
df_useful[['temp_no', 'temp_date_unit']] = df_useful['date'].str.extract(r'(\d+)\s+(\w{2})')

In [16]:
df_useful['days_passed'] = 0
df_useful['days_passed'][df_useful['temp_date_unit'] == "ημ"] = 1 * df_useful['temp_no'].astype(float)
df_useful['days_passed'][df_useful['temp_date_unit'] == "εβ"] = 7 * df_useful['temp_no'].astype(float)
df_useful['days_passed'][df_useful['temp_date_unit'] == "μή"] = 30 * df_useful['temp_no'].astype(float)

In [17]:
df_clean = df_useful[(4 <= df_useful['price_per_area']) & (df_useful['price_per_area'] <= 25) & 
                     (df_useful['price'] >= 100) & 
                     (df_useful['type'] != "Κτίριο") & 
                     (df_useful['dimos'] != "WRONG") & 
                     (df_useful['level_length'] <= 3) &
                     ~((df_useful['type'] != "Διαμέρισμα") & (df_useful['level_length'] != 1))]

In [18]:
df_clean.to_csv(f'xe_clean_{date_to_run}.csv', encoding='utf-8-sig', index=False)

In [19]:
print(f"Original: {len(df_useful)} rows.\nFiltered: {len(df_clean)} rows.\nRemoved: {len(df_useful) - len(df_clean)} rows.")

Original: 5817 rows.
Filtered: 5446 rows.
Removed: 371 rows.
